# Data Handling

The goal of this notebook is to inspect the incoming HTML data files the application will ingest. 
The data will be restructured into a pandas dataframe where we can illustrate how various rules can be applied to racer's timing data to deduce their grid position in the finals.

The grids of a race are not solely determined by lap time data. In total there are 3 factors. 

1. Lap times; the shorter lap times relative to other drivers results in greater overall grid position.


2. Overseer's context; the race overseer applies these rules or corrections so that the race data beyond raw timing data. Drivers in breech of regulations will face a time penalty which results in a lower overall grid position.


3. Novice Drivers; very inexperienced drivers may be forced to use the novice grid which is behind the main grid for safety. After a few races they can join the main grid or optionally waive their position on the main grid to use the novice grid. 


## Timing Data
The karts have transponders that detect when the driver completes a lap by passing a specified section on the race track. 
From this, the laptimes are easily extrapolated. 
The transponder therefore provides a one dimensional view of the racing data.
In isolation, the transponder timing data is fallable.
For instance, a driver may have artifically low lap times by taking illegal short cuts. 
Alternatively, the driver may drive in breech of regulations. For example, the combined weight of kart and driver is too low for the weight category. Additional context needs to be superimposed over the timing data to validate or correct it.

## Overseer's Context
The race overseer applies this additional context to the data. 
They currate it such that it tells a more complete story of the heats and races. 
The overseer recieves radio feedback from the marshals and flagman who may report various racing incidents.
The overseer can then respond to these racing incidents by applying time penalties, discounting laps or disqualifing drivers.
Furthermore, the overseer can communicate with marshels to use flags to communicate to drivers.
When karts return the post race/heat checks are performed.
The karts are weighed, wheels inspected, fuel is tested and kart nose cones are checked.
If there are any failed checks the driver is sanctioned and perhaps a penalty is applied to their time.

## Gridman's Manual Handling
The curated data is then published and the gridman is responsible to analysing it.
Often they determine each racers grid position by hand and record it with pen and paper.
This can be prone to human error and leave some drivers disgruntled where they feel they were not placed incorrectly.
Furthermore, while setting up the grid, some drivers will choose to waive their current position to start in the novice grid.

## Correcting for Novice Drivers
Some novice drivers who are transitioning to be experienced drivers can join the main grid.
They take their rightful position on the main grid, like experienced drivers.
Alternatively, they may opt to return to the novice grid and waive their position on the main grid.
In that case, they are still placed appropriately relative to other novice drivers. 
Or for very new drivers it is mandatory that they join the novice grid while aquireing sufficient experience.
For these reasons, it's useful to model the overall grid as being composed of two sub grids. 

1. The main grid; for experienced drivers


2. The novive grid; for less experienced drivers who need some more breathing space on track.


## Import Dependencies

In [1]:
import sys
from os import getcwd
from os.path import dirname, join
import pandas as pd

from IPython.display import Code


#jupiters kernel 
project_root = dirname(getcwd())
sys.path.append(project_root)

from kartingData.sample_data_api import load_heat1
from kartingData.prepare_data import prepare_dataframe

## Prepare Raw Data

High level overview of the sample data provided. 
The basic transponder data can be seen. Look at columns and data types and for any quirks in data structure.
1. Most columns should logically irrelivent to the grid algorithm, and can be removed.

2. Most Rows should be valid and represent a lap for a driver. We'll keep those.

3. Other rows signify race events such as warmup, start, finish. They NaN values generated stop us from casting easily. They will be removed.

4. The race overseer has filled in a Deleted column. Hopefully that captures most of the invalid data a driver produces.

Some commands will be used just to gain insight into the data. The preparation steps were first written in jupyter notebook, then moved to a module where the rest of the application could call them. We display the data preparation module in this notebook so its clear how the data was processed :) 


In [2]:
df=load_heat1()

def head_and_tail(df):
    #first 5 rows and last five rows
    return pd.concat([df.head(), df.tail()])

head_and_tail(df)

,#,No.,Name,Laps,Lead,Lap Tm,Spd,Elapsed Tm,Passing Tm,Hits,Strength,Noise,Photocell Time,Transponder,Backup Tx,Backup Passing Tm,Class,Deleted
0,1,NaN,Warmup Flag,NaN,NaN,NaN,NaN,NaN,10:18:54.484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,73.0,Ben MCCLOUGHRY,0.0,0.0,NaN,0.0,NaN,10:19:02.125,458.0,76.0,26.0,NaN,33879.0,0.0,NaN,Bam,No
2,3,27.0,Brogan MCDONALD,0.0,0.0,NaN,0.0,NaN,10:19:05.192,88.0,52.0,26.0,NaN,2742019.0,0.0,NaN,Bam,No
3,4,12.0,Tommy NAUGHTON HEAVEY,0.0,0.0,NaN,0.0,NaN,10:19:07.200,285.0,87.0,26.0,NaN,6754955.0,0.0,NaN,Bam,No
4,5,NaN,Green Flag,NaN,NaN,NaN,NaN,0.000,10:19:16.949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63,86.0,Alex REILLY (N),7.0,7.0,1:10.979,76.1,8:20.877,10:27:37.826,58.0,133.0,23.0,NaN,3432104.0,0.0,NaN,Bam,No
63,64,44.0,Arianne EGAN (N),6.0,7.0,1:37.072,55.6,8:27.103,10:27:44.052,126.0,136.0,23.0,NaN,1685552.0,0.0,NaN,Bam,No
64,65,34.0,Oscar CONNOLLY,7.0,7.0,1:12.352,74.6,8:35.904,10:27:52.853,65.0,128.0,21.0,NaN,91301.0,0.0,NaN,Bam,No
65,66,12.0,Tommy NAUGHTON HEAVEY,7.0,7.0,1:16.471,70.6,8:47.762,10:28:04.711,68.0,131.0,21.0,NaN,6754955.0,0.0,NaN,Bam,No
66,67,NaN,Run Stopped,NaN,NaN,NaN,NaN,8:55.272,10:28:12.221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info(memory_usage='Deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   #                  67 non-null     int64  
 1   No.                63 non-null     float64
 2   Name               67 non-null     object 
 3   Laps               63 non-null     float64
 4   Lead               63 non-null     float64
 5   Lap Tm             47 non-null     object 
 6   Spd                63 non-null     float64
 7   Elapsed Tm         58 non-null     object 
 8   Passing Tm         67 non-null     object 
 9   Hits               63 non-null     float64
 10  Strength           63 non-null     float64
 11  Noise              63 non-null     float64
 12  Photocell Time     0 non-null      float64
 13  Transponder        63 non-null     float64
 14  Backup Tx          63 non-null     float64
 15  Backup Passing Tm  0 non-null      float64
 16  Class              63 non-nu

In [4]:
df.describe()

,#,No.,Laps,Lead,Spd,Hits,Strength,Noise,Photocell Time,Transponder,Backup Tx,Backup Passing Tm
count,67.000000,63.000000,63.00000,63.000000,63.000000,63.000000,63.000000,63.000000,0.0,6.300000e+01,63.0,0.0
mean,34.000000,43.492063,3.52381,3.555556,57.136508,139.825397,136.349206,25.365079,NaN,2.783072e+06,0.0,NaN
std,19.485037,27.694109,2.17654,2.212704,33.939217,97.884649,21.515305,6.350772,NaN,2.234730e+06,0.0,NaN
min,1.000000,6.000000,0.00000,0.000000,0.000000,45.000000,52.000000,16.000000,NaN,3.387900e+04,0.0,NaN
25%,17.500000,19.500000,1.50000,1.500000,27.800000,78.000000,131.500000,23.000000,NaN,8.884265e+05,0.0,NaN
50%,34.000000,34.000000,3.00000,3.000000,75.000000,107.000000,135.000000,23.000000,NaN,2.742019e+06,0.0,NaN
75%,50.500000,69.500000,5.00000,5.500000,80.400000,129.500000,137.500000,29.000000,NaN,4.352596e+06,0.0,NaN
max,67.000000,86.000000,7.00000,7.000000,82.200000,458.000000,183.000000,36.000000,NaN,6.754955e+06,0.0,NaN


In [5]:
df.memory_usage(deep=True)

Index                 128
#                     536
No.                   536
Name                 4838
Laps                  536
Lead                  536
Lap Tm               3695
Spd                   536
Elapsed Tm           4055
Passing Tm           4623
Hits                  536
Strength              536
Noise                 536
Photocell Time        536
Transponder           536
Backup Tx             536
Backup Passing Tm     536
Class                3908
Deleted              3850
dtype: int64

In [12]:
#Code is stored in a module for easy re-use in wider application.
#The processing methods are important to see here, so we display the module (read-only)
Code("prepare_data.py")

def remove_unnecessary_columns(df):
    '''These columns don't contribute to grid algorithm'''
    columns_to_drop = ['#', 'Name', 'Lead', 'Spd', 'Hits',
                       'Strength', 'Noise', 'Transponder',
                       'Backup Tx', 'Class', 'Photocell Time',
                       'Elapsed Tm', 'Passing Tm', 'Backup Passing Tm']
    return df.drop(columns=columns_to_drop)

def cast_driver_number_and_laps_to_int(df):
    return df.astype({'No.': 'int16', 'Laps':'int16'})

def remove_deleted_rows_and_deleted_column(df):
    '''race overseer marked these to be deleted.'''
    deleted_rows=df.index[df['Deleted'] != 'No']
    return (df.drop(labels=deleted_rows).drop(columns='Deleted'))

def remove_zeroth_and_first_laps(df):
    '''#Laps 0,1 don't have timing data'''
    laps_0_1_rows=df.index[df['Laps'] < 2]
    return df.drop(labels=laps_0_1_rows)


def prepare_dataframe(df):
    '''#pipe the above utilities together to prepare raw data.'''
    prepped_data = (df
                    #minimise memory footprint
                    .pipe(remove_unnecessary_columns)
                    #data marked as deleted by Overseer
                    .pipe(remove_deleted_rows_and_deleted_column)
                    #easier on the eyes and memory.
                    .pipe(cast_driver_number_and_laps_to_int)
                    #these don't have timing data? Funny business rule here?
                    .pipe(remove_zeroth_and_first_laps)
                   )
    return prepped_data

In [7]:
df=prepare_dataframe(df)
df.head()

,No.,Laps,Lap Tm
18,6,2,1:07.453
19,66,2,1:08.257
20,73,2,1:08.255
21,27,2,1:09.138
22,86,2,1:10.629


In [8]:
df.info(memory_usage='Deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 18 to 65
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   No.     47 non-null     int16 
 1   Laps    47 non-null     int16 
 2   Lap Tm  47 non-null     object
dtypes: int16(2), object(1)
memory usage: 940.0+ bytes


In [9]:
df.memory_usage(deep=True)

Index      376
No.         94
Laps        94
Lap Tm    3055
dtype: int64

### Now we've reduced the heat data to the bear essentials.